In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
import pandas as pd

In [7]:
X_train = pd.read_csv('../data/processed/X_train', header=0, index_col=0)
X_test = pd.read_csv('../data/processed/X_test', header=0, index_col=0)
y_train = pd.read_csv('../data/processed/y_train', header=0, index_col=0)
y_test = pd.read_csv('../data/processed/y_test', header=0, index_col=0)

In [8]:
model = 'Random Forest Classfier'
rf = RandomForestClassifier(n_estimators=20, random_state=42,max_depth=5)
rf.fit(X_train,y_train)
rf_predicted = rf.predict(X_test)
rf_conf_matrix = confusion_matrix(y_test, rf_predicted)
rf_acc_score = accuracy_score(y_test, rf_predicted)
print("confusion matrix")
print(rf_conf_matrix)
print("\n")
print("Accuracy of Random Forest:",rf_acc_score*100,'\n')
print(classification_report(y_test,rf_predicted))

confusion matrix
[[33  3]
 [ 4 20]]


Accuracy of Random Forest: 88.33333333333333 

              precision    recall  f1-score   support

           0       0.89      0.92      0.90        36
           1       0.87      0.83      0.85        24

    accuracy                           0.88        60
   macro avg       0.88      0.88      0.88        60
weighted avg       0.88      0.88      0.88        60



/var/folders/wh/6llm7dq16rvbwjb_yc0kfvvr0000gn/T/ipykernel_27315/1203641609.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train,y_train)
